<a href="https://colab.research.google.com/github/Rubnserrano/DataScience/blob/main/08_curso_tensorflow_certificate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NLP first steps (exciting!!!)

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-55120588-781e-8f7f-23c8-95ca26f9304b)


In [2]:
!wget https://raw.githubusercontent.com/Rubnserrano/TF-Certification/main/extra/helper_functions.py
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2022-11-23 12:42:22--  https://raw.githubusercontent.com/Rubnserrano/TF-Certification/main/extra/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10116 (9.9K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   9.88K  --.-KB/s    in 0s      

2022-11-23 12:42:22 (75.2 MB/s) - ‘helper_functions.py’ saved [10116/10116]



In [3]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"
unzip_data("nlp_getting_started.zip")

--2022-11-23 12:42:34--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.24.128, 172.217.194.128, 142.251.10.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.24.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-11-23 12:42:35 (147 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
!ls

helper_functions.py	 __pycache__  sample_submission.csv  train.csv
nlp_getting_started.zip  sample_data  test.csv


In [5]:
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.tail()

,id,keyword,location,text,target
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1
7612,10873,NaN,NaN,The Latest: More Homes Razed by Northern Calif...,1


In [6]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
import numpy as np
print(train_df.target.value_counts())

0    4342
1    3271
Name: target, dtype: int64


In [8]:
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [9]:
train_df_shuffled.columns

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

In [10]:
import random

train_df_shuffled.text[1]

'Forest fire near La Ronge Sask. Canada'

In [11]:
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
If you ever think you running out of choices in life rembr there's that kid that has no choice but wear a suicide bomb vest

---

Target: 0 (not real disaster)
Text:
@ohmyloz @RondaRousey who is bagging her body ? She's smoking hot ??

---

Target: 1 (real disaster)
Text:
#Sismo DETECTADO #JapÌ_n 06:32:43 Miyagi Estimated seismic intensity 0 JST #??

---

Target: 0 (not real disaster)
Text:
I'm traumatised???? @megancoopy @laurathorne97 http://t.co/MeSqTVdu63

---

Target: 0 (not real disaster)
Text:
By the grace of GOD I survived the 2am shift and im not that tired.

---



In [12]:
from sklearn.model_selection import train_test_split

In [13]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                            train_df_shuffled['target'].to_numpy(),
                                                                            test_size = 0.1,
                                                                            random_state=42)

In [14]:
print(f"Train sentences: {len(train_sentences)} and train labels: {len(train_labels)}")
print(f"Train sentences: {len(val_sentences)} and train labels: {len(val_labels)}")

Train sentences: 6851 and train labels: 6851
Train sentences: 762 and train labels: 762


In [15]:
train_sentences[:5], train_labels[:5]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
       dtype=object), array([0, 0, 1, 0, 0]))

The TextVectorization layer takes the following parameters:

* max_tokens - The maximum number of words in your vocabulary (e.g. 20000 or the number of unique words in your text), includes a value for OOV (out of vocabulary) tokens.
* standardize - Method for standardizing text. Default is "lower_and_strip_punctuation" which lowers text and removes all punctuation marks.
* split - How to split text, default is "whitespace" which splits on spaces.
* ngrams - How many words to contain per token split, for example, ngrams=2 splits tokens into continuous sequences of 2.
* output_mode - How to output tokens, can be "int" (integer mapping), "binary" (one-hot encoding), "count" or "tf-idf". See documentation for more.
* output_sequence_length - Length of tokenized sequence to output. For example, if output_sequence_length=150, all tokenized sequences will be 150 tokens long.
* pad_to_max_tokens - Defaults to False, if True, the output feature axis will be padded to max_tokens even if the number of unique tokens in the vocabulary is less than max_tokens. Only valid in certain modes, see docs for more.

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [21]:
text_vectorizer = TextVectorization(max_tokens = None,
                                    standardize = 'lower_and_strip_punctuation' ,
                                    split = 'whitespace',
                                    ngrams = None,
                                    output_mode = 'int' ,
                                    output_sequence_length = None)

In [22]:
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [24]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens =  max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length = max_length)

In [25]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [28]:
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [39]:
import random
random_sentence = random.choice(train_sentences)
print(f'This is the real sentence: \n {random_sentence} \n')
tokenized_sentence = text_vectorizer([random_sentence])
print(f'This is the tokenized sentence: \n {tokenized_sentence}')

This is the real sentence: 
 @tweetingLew @tersestuff 

Notley flattened Harper IN HIS Heartland
Harper Imported tens of thousands of TFW slaves COST ALBERTANS JOBS 

This is the tokenized sentence: 
 [[7290 7707    1  485 3788    4   79    1 3788 3748 4447    6  981    6
  7693]]


In [42]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f'Number of words in vocab {len(words_in_vocab)}')
print(f'Top 5 words: {top_5_words}')
print(f'Bottom 5 words: {bottom_5_words}')

Number of words in vocab 10000
Top 5 words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


#Creating an Embedding using an Embedding Layer

In [44]:
tf.random.set_seed(42)
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 128,
                             embeddings_initializer = 'uniform',
                             input_length = max_length,
                             name = 'embedding_1')

In [46]:
random_sentence = random.choice(train_sentences)
print(f'Original sentence:\n {random_sentence}\n')
sample_embedded = embedding(text_vectorizer([random_sentence]))
print(f'Embedded sentence: \n {sample_embedded} \n')

Original sentence:
 Jacksonville Busines FedEx stops shipping potential bioterror pathogens http://t.co/sHzsYmaUSi

Embedded sentence: 
 [[[-0.01699571  0.03074422 -0.02173766 ...  0.02035892 -0.01679568
    0.01461729]
  [ 0.03977952 -0.03782602 -0.03646283 ...  0.00236253  0.03332629
    0.02803668]
  [-0.04178156  0.01266341  0.00283604 ... -0.04867874 -0.01714375
    0.04544358]
  ...
  [ 0.01645621 -0.00589932 -0.01471175 ... -0.02511839  0.00912381
   -0.00024097]
  [ 0.01645621 -0.00589932 -0.01471175 ... -0.02511839  0.00912381
   -0.00024097]
  [ 0.01645621 -0.00589932 -0.01471175 ... -0.02511839  0.00912381
   -0.00024097]]] 



In [ ]:
sample_embbeded[0][0]
